In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
from datetime import *
from sklearn import linear_model
from sklearn.preprocessing import *
from sklearn import ensemble
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import xgboost as xgb
from sklearn import metrics
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import GridSearchCV,cross_val_score  
import matplotlib.pylab as plt 

In [24]:
#首先导入滑窗训练集数据进行线下数据集划分
train_df = pd.read_csv('chusai_huachuang_2daysHistory_train_df.csv')
test_df = pd.read_csv('chusai_huachuang_2daysHistory_test_df.csv')

fea = [
        'context_page_id_1', 'context_page_id_2', 'context_page_id_3', 'context_page_id_4', 
       'item_pv_level_low', 'shop_star_level_loss', 'user_gender_id', 'user_occupation_id', 'user_star_level',
       'hour', 'day_hour', #'weekday',
       'user_age_level', 'item_sales_level', 'item_brand_id', 'item_city_id',
       'item_collected_level', 'item_price_level', 'item_pv_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
       'match_category_proportion', 'match_property_proportion',
       'predict_category_number', 'predict_property_number', 'category_number', 'property_number', 
       'isFirstCategoryIn', 'isLastCategoryIn', 
       
        'lastOneHour_sameItem_count', 'lastOneHour_sameFirstCategory_count', 'lastOneHour_sameLastCategory_count', 
       'lastOneHour_sameBrand_count', 'lastOneHour_sameShop_count', 
    
        'lastOneDay_sameItem_count', 'lastOneDay_sameFirstCategory_count', 'lastOneDay_sameLastCategory_count', 
       'lastOneDay_sameBrand_count', 'lastOneDay_sameShop_count', 
        
       'isLastOneHour_firstClickItem', 
       'is_history_sameItem', 'is_history_sameFirstCategory', 'is_history_sameLastCategory',
       'is_highProportion_brand', 'is_highSale_brand', 
       'isHigh_shop_score_service', 'isHigh_shop_score_delivery',
       'isHigh_shop_score_description', 'isHigh_shop_review_positive_rate',
       'is_history_sameBrand', 'is_history_sameShop', #'is_history_firstClickItem',
       'shop_review_positive_rate_diff', 'shop_score_service_diff', 'shop_score_delivery_diff', 'shop_score_description_diff', 
        'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber', 
       'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber',
       
#         'hour_converse_smooth_rate',
    
        'user_id_total_number', 'item_brand_id_total_number', 'item_id_total_number', 
       'shop_id_total_number', 'item_city_id_total_number', 
       'real_first_category_total_number', 'real_last_category_total_number', 
       'user_item_total_number', 'user_brand_total_number', 'user_shop_total_number',
        'user_city_total_number', 'user_first_category_total_number', 'user_last_category_total_number',
        
        'user_id_buy_number', 'item_brand_id_buy_number', 'item_id_buy_number', 
       'shop_id_buy_number', 'item_city_id_buy_number', 
       'real_first_category_buy_number', 'real_last_category_buy_number', 
       'user_item_buy_number', 'user_brand_buy_number', 'user_shop_buy_number',
        'user_city_buy_number', 'user_first_category_buy_number', 'user_last_category_buy_number', 
        
        'user_id_converse_smooth_rate', 'item_brand_id_converse_smooth_rate', 'item_id_converse_smooth_rate', 
       'shop_id_converse_smooth_rate', 'item_city_id_converse_smooth_rate', 
       'real_first_category_converse_smooth_rate', 'real_last_category_converse_smooth_rate', 
       'user_item_converse_smooth_rate', 'user_brand_converse_smooth_rate', 'user_shop_converse_smooth_rate',
        'user_city_converse_smooth_rate', 'user_first_category_converse_smooth_rate', 'user_last_category_converse_smooth_rate', 
       
#         'user_item_price_converse_smooth_rate', 'user_item_sales_converse_smooth_rate', 
#         'user_item_price_total_number', 'user_item_sales_total_number',
#         'user_item_price_buy_number', 'user_item_sales_buy_number',
        
#         'userItem_lastClickDeltaTime', 'userBrand_lastClickDeltaTime', 'userShop_lastClickDeltaTime', 
#         'userFirstCategory_lastClickDeltaTime', 'userLastCategory_lastClickDeltaTime', 
    
#     'user_gender_item_converse_smooth_rate', 'user_gender_brand_converse_smooth_rate', 'user_gender_shop_converse_smooth_rate', 
#     'user_age_item_converse_smooth_rate', 'user_age_brand_converse_smooth_rate', 'user_age_shop_converse_smooth_rate', 
#     'user_occupation_item_converse_smooth_rate', 'user_occupation_brand_converse_smooth_rate', 'user_occupation_shop_converse_smooth_rate', 
#     'user_star_item_converse_smooth_rate', 'user_star_brand_converse_smooth_rate', 'user_star_shop_converse_smooth_rate', 
    
#     'user_gender_item_total_number', 'user_gender_brand_total_number', 'user_gender_shop_total_number', 
#     'user_age_item_total_number', 'user_age_brand_total_number', 'user_age_shop_total_number', 
#     'user_occupation_item_total_number', 'user_occupation_brand_total_number', 'user_occupation_shop_total_number',
#     'user_star_item_total_number', 'user_star_brand_total_number', 'user_star_shop_total_number', 
    
#      'user_gender_item_buy_number', 'user_gender_brand_buy_number', 'user_gender_shop_buy_number', 
#       'user_age_item_buy_number', 'user_age_brand_buy_number', 'user_age_shop_buy_number', 
#     'user_occupation_item_buy_number', 'user_occupation_brand_buy_number', 'user_occupation_shop_buy_number',
#     'user_star_item_buy_number', 'user_star_brand_buy_number',  'user_star_shop_buy_number',
    
        'history_sameItem_count', 'history_sameFirstCategory_count', 'history_sameLastCategory_count',
        'history_sameBrand_count', 'history_sameShop_count'
      ]

train_dataset1_x = train_df[fea][(train_df.day >= 20) & (train_df.day <= 23)]
train_dataset1_y = train_df['is_trade'][(train_df.day >= 20) & (train_df.day <= 23)]
test_dataset1_x = train_df[fea][(train_df.day == 24)]
test_dataset1_y = train_df['is_trade'][(train_df.day == 24)]

train_dataset_x = train_df[fea][(train_df.day >= 20) & (train_df.day <= 24)]
train_dataset_y = train_df['is_trade'][(train_df.day >= 20) & (train_df.day <= 24)]
test_dataset_x = test_df[fea]
print(len(train_dataset_x.columns))
print(len(test_dataset_x.columns))


109
109


In [38]:
#开始进行xgb模型训练
xgb_train_dataset1 = xgb.DMatrix(train_dataset1_x,label = train_dataset1_y)
xgb_test_dataset1 = xgb.DMatrix(test_dataset1_x, label = test_dataset1_y)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.5,
        'min_child_weight':10,
        'max_depth':5,
        'lambda':11,
        'subsample':0.85,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.01,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

offline_watchlist = [(xgb_train_dataset1,'train'), (xgb_test_dataset1, 'val')]
offline_model = xgb.train(cv_params, xgb_train_dataset1, num_boost_round=1250, evals=offline_watchlist)


[0]	train-logloss:0.683928	val-logloss:0.683832
[1]	train-logloss:0.674892	val-logloss:0.674706
[2]	train-logloss:0.66604	val-logloss:0.66574
[3]	train-logloss:0.657353	val-logloss:0.656919
[4]	train-logloss:0.648834	val-logloss:0.648294
[5]	train-logloss:0.640478	val-logloss:0.639829
[6]	train-logloss:0.632278	val-logloss:0.631541
[7]	train-logloss:0.624235	val-logloss:0.623384
[8]	train-logloss:0.616341	val-logloss:0.615409
[9]	train-logloss:0.608593	val-logloss:0.607565
[10]	train-logloss:0.600989	val-logloss:0.599881
[11]	train-logloss:0.59352	val-logloss:0.592316
[12]	train-logloss:0.586186	val-logloss:0.584889
[13]	train-logloss:0.578984	val-logloss:0.577609
[14]	train-logloss:0.571915	val-logloss:0.570488
[15]	train-logloss:0.564966	val-logloss:0.56347
[16]	train-logloss:0.55814	val-logloss:0.556593
[17]	train-logloss:0.551433	val-logloss:0.549819
[18]	train-logloss:0.544846	val-logloss:0.543131
[19]	train-logloss:0.538368	val-logloss:0.536596
[20]	train-logloss:0.531999	val-log

[167]	train-logloss:0.152871	val-logloss:0.147305
[168]	train-logloss:0.152035	val-logloss:0.146453
[169]	train-logloss:0.151208	val-logloss:0.1456
[170]	train-logloss:0.150391	val-logloss:0.144792
[171]	train-logloss:0.149586	val-logloss:0.144516
[172]	train-logloss:0.148789	val-logloss:0.143686
[173]	train-logloss:0.148003	val-logloss:0.142865
[174]	train-logloss:0.147225	val-logloss:0.142096
[175]	train-logloss:0.146457	val-logloss:0.141296
[176]	train-logloss:0.145701	val-logloss:0.140517
[177]	train-logloss:0.144955	val-logloss:0.139735
[178]	train-logloss:0.144215	val-logloss:0.138959
[179]	train-logloss:0.143486	val-logloss:0.138219
[180]	train-logloss:0.142766	val-logloss:0.137566
[181]	train-logloss:0.142055	val-logloss:0.136929
[182]	train-logloss:0.141354	val-logloss:0.136555
[183]	train-logloss:0.140658	val-logloss:0.135844
[184]	train-logloss:0.139971	val-logloss:0.135133
[185]	train-logloss:0.139294	val-logloss:0.134414
[186]	train-logloss:0.138626	val-logloss:0.133729
[1

[332]	train-logloss:0.094355	val-logloss:0.090666
[333]	train-logloss:0.094255	val-logloss:0.090617
[334]	train-logloss:0.094156	val-logloss:0.090556
[335]	train-logloss:0.094058	val-logloss:0.090459
[336]	train-logloss:0.093961	val-logloss:0.090426
[337]	train-logloss:0.093865	val-logloss:0.090312
[338]	train-logloss:0.09377	val-logloss:0.090195
[339]	train-logloss:0.093676	val-logloss:0.090083
[340]	train-logloss:0.093585	val-logloss:0.089977
[341]	train-logloss:0.093494	val-logloss:0.08987
[342]	train-logloss:0.093403	val-logloss:0.089772
[343]	train-logloss:0.093313	val-logloss:0.089717
[344]	train-logloss:0.093223	val-logloss:0.089611
[345]	train-logloss:0.093136	val-logloss:0.089506
[346]	train-logloss:0.093051	val-logloss:0.089412
[347]	train-logloss:0.092967	val-logloss:0.089318
[348]	train-logloss:0.092882	val-logloss:0.089225
[349]	train-logloss:0.0928	val-logloss:0.089141
[350]	train-logloss:0.092718	val-logloss:0.089105
[351]	train-logloss:0.092638	val-logloss:0.089008
[352

[497]	train-logloss:0.087267	val-logloss:0.084206
[498]	train-logloss:0.087252	val-logloss:0.084191
[499]	train-logloss:0.087237	val-logloss:0.084173
[500]	train-logloss:0.087221	val-logloss:0.084157
[501]	train-logloss:0.087206	val-logloss:0.084129
[502]	train-logloss:0.08719	val-logloss:0.084111
[503]	train-logloss:0.087177	val-logloss:0.0841
[504]	train-logloss:0.08716	val-logloss:0.084085
[505]	train-logloss:0.087146	val-logloss:0.084066
[506]	train-logloss:0.087131	val-logloss:0.084053
[507]	train-logloss:0.087117	val-logloss:0.084038
[508]	train-logloss:0.087103	val-logloss:0.084035
[509]	train-logloss:0.087088	val-logloss:0.084024
[510]	train-logloss:0.087076	val-logloss:0.084012
[511]	train-logloss:0.087062	val-logloss:0.084005
[512]	train-logloss:0.087049	val-logloss:0.08399
[513]	train-logloss:0.087036	val-logloss:0.083973
[514]	train-logloss:0.087024	val-logloss:0.08396
[515]	train-logloss:0.087012	val-logloss:0.083949
[516]	train-logloss:0.086999	val-logloss:0.083938
[517]	

[662]	train-logloss:0.085714	val-logloss:0.083028
[663]	train-logloss:0.085707	val-logloss:0.083023
[664]	train-logloss:0.085703	val-logloss:0.083015
[665]	train-logloss:0.085696	val-logloss:0.083012
[666]	train-logloss:0.085689	val-logloss:0.083012
[667]	train-logloss:0.085683	val-logloss:0.083011
[668]	train-logloss:0.085678	val-logloss:0.083006
[669]	train-logloss:0.085672	val-logloss:0.083
[670]	train-logloss:0.085666	val-logloss:0.082997
[671]	train-logloss:0.085659	val-logloss:0.082995
[672]	train-logloss:0.085652	val-logloss:0.082994
[673]	train-logloss:0.085647	val-logloss:0.082994
[674]	train-logloss:0.085642	val-logloss:0.082992
[675]	train-logloss:0.085636	val-logloss:0.08299
[676]	train-logloss:0.085632	val-logloss:0.082982
[677]	train-logloss:0.085626	val-logloss:0.082983
[678]	train-logloss:0.085619	val-logloss:0.082978
[679]	train-logloss:0.085616	val-logloss:0.082972
[680]	train-logloss:0.085612	val-logloss:0.082971
[681]	train-logloss:0.085608	val-logloss:0.082968
[682

[827]	train-logloss:0.084869	val-logloss:0.082704
[828]	train-logloss:0.084865	val-logloss:0.082702
[829]	train-logloss:0.084861	val-logloss:0.082702
[830]	train-logloss:0.084858	val-logloss:0.082703
[831]	train-logloss:0.084855	val-logloss:0.082704
[832]	train-logloss:0.084851	val-logloss:0.082706
[833]	train-logloss:0.084846	val-logloss:0.082705
[834]	train-logloss:0.084841	val-logloss:0.082701
[835]	train-logloss:0.084836	val-logloss:0.082701
[836]	train-logloss:0.084831	val-logloss:0.082699
[837]	train-logloss:0.084827	val-logloss:0.082695
[838]	train-logloss:0.084822	val-logloss:0.082695
[839]	train-logloss:0.084818	val-logloss:0.082697
[840]	train-logloss:0.084814	val-logloss:0.082698
[841]	train-logloss:0.084811	val-logloss:0.082696
[842]	train-logloss:0.084805	val-logloss:0.082694
[843]	train-logloss:0.084802	val-logloss:0.082694
[844]	train-logloss:0.084796	val-logloss:0.082692
[845]	train-logloss:0.08479	val-logloss:0.082694
[846]	train-logloss:0.084787	val-logloss:0.082696
[

[992]	train-logloss:0.084194	val-logloss:0.082638
[993]	train-logloss:0.084187	val-logloss:0.082641
[994]	train-logloss:0.084184	val-logloss:0.082645
[995]	train-logloss:0.084178	val-logloss:0.082644
[996]	train-logloss:0.084176	val-logloss:0.082645
[997]	train-logloss:0.084173	val-logloss:0.082652
[998]	train-logloss:0.084171	val-logloss:0.082658
[999]	train-logloss:0.084169	val-logloss:0.082664
[1000]	train-logloss:0.084165	val-logloss:0.082664
[1001]	train-logloss:0.084162	val-logloss:0.082664
[1002]	train-logloss:0.084159	val-logloss:0.082664
[1003]	train-logloss:0.084157	val-logloss:0.082671
[1004]	train-logloss:0.084154	val-logloss:0.082671
[1005]	train-logloss:0.084152	val-logloss:0.082679
[1006]	train-logloss:0.084146	val-logloss:0.082679
[1007]	train-logloss:0.084143	val-logloss:0.082677
[1008]	train-logloss:0.084138	val-logloss:0.082683
[1009]	train-logloss:0.084133	val-logloss:0.082676
[1010]	train-logloss:0.084129	val-logloss:0.082683
[1011]	train-logloss:0.084125	val-loglo

[1154]	train-logloss:0.083608	val-logloss:0.082622
[1155]	train-logloss:0.083607	val-logloss:0.082619
[1156]	train-logloss:0.083606	val-logloss:0.082626
[1157]	train-logloss:0.083601	val-logloss:0.082628
[1158]	train-logloss:0.083599	val-logloss:0.082627
[1159]	train-logloss:0.083593	val-logloss:0.082625
[1160]	train-logloss:0.08359	val-logloss:0.082624
[1161]	train-logloss:0.083586	val-logloss:0.08262
[1162]	train-logloss:0.083584	val-logloss:0.082615
[1163]	train-logloss:0.083582	val-logloss:0.082622
[1164]	train-logloss:0.083577	val-logloss:0.082619
[1165]	train-logloss:0.083575	val-logloss:0.082609
[1166]	train-logloss:0.08357	val-logloss:0.082609
[1167]	train-logloss:0.083568	val-logloss:0.082609
[1168]	train-logloss:0.083563	val-logloss:0.082606
[1169]	train-logloss:0.083561	val-logloss:0.082614
[1170]	train-logloss:0.083558	val-logloss:0.082612
[1171]	train-logloss:0.083555	val-logloss:0.082614
[1172]	train-logloss:0.083552	val-logloss:0.082608
[1173]	train-logloss:0.083547	val-

In [39]:
#输出线下训练的xgb模型的重要性
feat_imp = pd.Series(offline_model.get_fscore()).sort_values(ascending=False)
print(feat_imp.head(100))


hour                                        1137
item_id_converse_smooth_rate                1069
shop_score_delivery                         1051
shop_id_converse_smooth_rate                1035
item_sales_level                             805
item_brand_id_converse_smooth_rate           763
shop_score_delivery_diff                     710
shop_score_description_diff                  658
user_star_level                              648
property_number                              644
shop_score_service                           644
real_first_category_converse_smooth_rate     642
item_city_id_converse_smooth_rate            633
shop_score_service_diff                      631
user_age_level                               598
shop_score_description                       582
shop_id_total_number                         568
shop_review_positive_rate_diff               524
item_brand_id                                521
item_id_total_number                         516
item_brand_id_total_

In [40]:
#计算线下误差
def calError(col1, col2):
    error = metrics.log_loss(col1, col2)
    return error

xgb_test = xgb.DMatrix(test_dataset1_x)
test_result = offline_model.predict(xgb_test)
dataset1_error = calError(test_dataset1_y.values, test_result)
print('dataset1_error : ', dataset1_error)

dataset1_error :  0.0825799907665


In [41]:
#进行正式模型训练
xgb_train = xgb.DMatrix(train_dataset_x, label=train_dataset_y)
xgb_test = xgb.DMatrix(test_dataset_x)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.5,
        'min_child_weight':10,
        'max_depth':5,
        'lambda':11,
        'subsample':0.85,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.01,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

watchlist = [(xgb_train,'train')]
model = xgb.train(cv_params, xgb_train, num_boost_round=1250, evals=watchlist)


[0]	train-logloss:0.683918
[1]	train-logloss:0.67487
[2]	train-logloss:0.665998
[3]	train-logloss:0.657302
[4]	train-logloss:0.648774
[5]	train-logloss:0.640405
[6]	train-logloss:0.632194
[7]	train-logloss:0.624141
[8]	train-logloss:0.616237
[9]	train-logloss:0.608482
[10]	train-logloss:0.600866
[11]	train-logloss:0.593392
[12]	train-logloss:0.58605
[13]	train-logloss:0.578841
[14]	train-logloss:0.571761
[15]	train-logloss:0.564802
[16]	train-logloss:0.55797
[17]	train-logloss:0.551254
[18]	train-logloss:0.544656
[19]	train-logloss:0.538174
[20]	train-logloss:0.531798
[21]	train-logloss:0.525535
[22]	train-logloss:0.519374
[23]	train-logloss:0.513316
[24]	train-logloss:0.507358
[25]	train-logloss:0.5015
[26]	train-logloss:0.49574
[27]	train-logloss:0.490074
[28]	train-logloss:0.484501
[29]	train-logloss:0.479015
[30]	train-logloss:0.473621
[31]	train-logloss:0.468312
[32]	train-logloss:0.463085
[33]	train-logloss:0.457944
[34]	train-logloss:0.452885
[35]	train-logloss:0.447906
[36]	tra

[288]	train-logloss:0.099503
[289]	train-logloss:0.099321
[290]	train-logloss:0.099144
[291]	train-logloss:0.098969
[292]	train-logloss:0.098796
[293]	train-logloss:0.098627
[294]	train-logloss:0.098456
[295]	train-logloss:0.098289
[296]	train-logloss:0.098125
[297]	train-logloss:0.097963
[298]	train-logloss:0.097802
[299]	train-logloss:0.097645
[300]	train-logloss:0.097489
[301]	train-logloss:0.097334
[302]	train-logloss:0.097182
[303]	train-logloss:0.097032
[304]	train-logloss:0.096884
[305]	train-logloss:0.096739
[306]	train-logloss:0.096594
[307]	train-logloss:0.096452
[308]	train-logloss:0.096311
[309]	train-logloss:0.096173
[310]	train-logloss:0.096037
[311]	train-logloss:0.095902
[312]	train-logloss:0.095768
[313]	train-logloss:0.095636
[314]	train-logloss:0.095506
[315]	train-logloss:0.095378
[316]	train-logloss:0.095252
[317]	train-logloss:0.095129
[318]	train-logloss:0.095005
[319]	train-logloss:0.094883
[320]	train-logloss:0.094762
[321]	train-logloss:0.094643
[322]	train-lo

[572]	train-logloss:0.085579
[573]	train-logloss:0.08557
[574]	train-logloss:0.085563
[575]	train-logloss:0.085554
[576]	train-logloss:0.085543
[577]	train-logloss:0.085536
[578]	train-logloss:0.085528
[579]	train-logloss:0.085517
[580]	train-logloss:0.085509
[581]	train-logloss:0.085502
[582]	train-logloss:0.085493
[583]	train-logloss:0.085486
[584]	train-logloss:0.085478
[585]	train-logloss:0.085471
[586]	train-logloss:0.085464
[587]	train-logloss:0.085455
[588]	train-logloss:0.085448
[589]	train-logloss:0.08544
[590]	train-logloss:0.085433
[591]	train-logloss:0.085424
[592]	train-logloss:0.085417
[593]	train-logloss:0.08541
[594]	train-logloss:0.085402
[595]	train-logloss:0.085392
[596]	train-logloss:0.085385
[597]	train-logloss:0.085376
[598]	train-logloss:0.085367
[599]	train-logloss:0.08536
[600]	train-logloss:0.085352
[601]	train-logloss:0.085346
[602]	train-logloss:0.085338
[603]	train-logloss:0.085331
[604]	train-logloss:0.085324
[605]	train-logloss:0.085317
[606]	train-loglos

[856]	train-logloss:0.084027
[857]	train-logloss:0.084023
[858]	train-logloss:0.084018
[859]	train-logloss:0.084014
[860]	train-logloss:0.084009
[861]	train-logloss:0.084007
[862]	train-logloss:0.084003
[863]	train-logloss:0.083998
[864]	train-logloss:0.083992
[865]	train-logloss:0.083987
[866]	train-logloss:0.083983
[867]	train-logloss:0.083978
[868]	train-logloss:0.083974
[869]	train-logloss:0.083971
[870]	train-logloss:0.083968
[871]	train-logloss:0.083963
[872]	train-logloss:0.08396
[873]	train-logloss:0.083955
[874]	train-logloss:0.08395
[875]	train-logloss:0.083947
[876]	train-logloss:0.083943
[877]	train-logloss:0.083939
[878]	train-logloss:0.083935
[879]	train-logloss:0.083933
[880]	train-logloss:0.08393
[881]	train-logloss:0.083928
[882]	train-logloss:0.083926
[883]	train-logloss:0.083923
[884]	train-logloss:0.083919
[885]	train-logloss:0.083916
[886]	train-logloss:0.083913
[887]	train-logloss:0.083907
[888]	train-logloss:0.083903
[889]	train-logloss:0.083899
[890]	train-loglo

[1135]	train-logloss:0.083043
[1136]	train-logloss:0.08304
[1137]	train-logloss:0.083038
[1138]	train-logloss:0.083037
[1139]	train-logloss:0.083032
[1140]	train-logloss:0.083028
[1141]	train-logloss:0.083026
[1142]	train-logloss:0.083024
[1143]	train-logloss:0.083021
[1144]	train-logloss:0.083018
[1145]	train-logloss:0.083015
[1146]	train-logloss:0.083013
[1147]	train-logloss:0.083008
[1148]	train-logloss:0.083003
[1149]	train-logloss:0.082999
[1150]	train-logloss:0.082996
[1151]	train-logloss:0.082992
[1152]	train-logloss:0.082988
[1153]	train-logloss:0.082985
[1154]	train-logloss:0.08298
[1155]	train-logloss:0.082977
[1156]	train-logloss:0.082973
[1157]	train-logloss:0.082969
[1158]	train-logloss:0.082965
[1159]	train-logloss:0.082962
[1160]	train-logloss:0.08296
[1161]	train-logloss:0.082955
[1162]	train-logloss:0.082952
[1163]	train-logloss:0.082949
[1164]	train-logloss:0.082944
[1165]	train-logloss:0.082942
[1166]	train-logloss:0.082937
[1167]	train-logloss:0.082936
[1168]	train-

In [42]:
#predict test set
test_df['predicted_score'] = model.predict(xgb_test)


In [43]:
print(len(test_df[['instance_id', 'predicted_score']][test_df.predicted_score > 0.1]))
print(test_df['predicted_score'].mean())
print(len(test_df['predicted_score']))


53
0.016331594437360764
42888


In [ ]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../data/chusai/submit/%s.txt' % fileName, sep=' ', header=True, index=False)

print(test_df[['instance_id', 'predicted_score']].head(10))
exportResult(test_df[['instance_id', 'predicted_score']], 'chusai_xgb_4_19_wen')
